In [47]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerRegularPolyCollection # for legend change color
# from matplotlib import rcParams, matplotlib_fname
# # set the font to support Chinese characters
# rcParams['font.sans-serif'] = ['SimHei']
# # set the font size
# rcParams['font.size'] = 12
# # avoid using unicode minus sign
# rcParams['axes.unicode_minus'] = False

import seaborn as sns
import os
import sys

# 获取当前文件所在的文件夹路径
if "__file__" in globals():
    # 获取__file__变量的值
    file_path = __file__
    # 获取当前文件所在的文件夹路径
    dir_now = os.path.dirname(file_path)
else:
    # 获取当前工作目录
    dir_now = os.getcwd()
sys.path.append(os.path.join(dir_now, '../../'))
sys.path.append(os.path.join(dir_now, '../'))

from NNutils import *
from amm_methods import *
import pickle

marker_list = ["s", "o", "v", "*", "1", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d", "3", "4", "8", "p", "h", "H", "<", ">", "d", "1", "v", "o", "s", "*", "+", "x", "D", "|", "^", "2", "P", "X", "d",]
colors_list = ['#F27970', '#BB9727', '#54B345', '#32B897', '#05B9E2',
              '#8983BF', '#C76DA2', '#FFBE7A', '#82B0D2', '#BEB8DC',
              '#E7DAD2', '#999999']
cmap_list = ['Blues', 'Reds', 'Greens', 'Purples', 'Greys', 'Oranges']
cmap_max_list = [(0.05636293733179547, 0.34963475586312953, 0.6367550941945406, 1.0), 
                 (0.4725720876585929, 0.016608996539792385, 0.059838523644752006, 1.0), 
                 (0.034986543637062675, 0.45743944636678197, 0.19753940792003075, 1.0), 
                 (0.2703114186851211, 0.043183391003460206, 0.5101268742791234, 1.0), 
                    (0.4, 0.4, 0.4, 1.0),
                 (0.5412226066897348, 0.16955017301038064, 0.014579008073817762, 1.0)]


/shared/conda/envs/bolt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



In [48]:
def is_even(num):
    return True if num % 2 == 0 else False
def is_odd(num):
    return True if num % 2 == 1 else False

## 分割阈值

In [49]:
linear_name = 'etl1'
feedback_bits = 256
linear_name_full = "ex_linear1"
method = METHOD_MITHRAL
ncodebooks = 64 # max:64
ncentroids = [128, 256, 512, 1024, 2048]
D = 64
train_sam_num = 1000 # 训练集样本数
test_sam_num = 1000
i_cb = 53 # 选择第几个codebook
i_ncentroids = 0 # 选择第几个ncentroids


In [50]:
AMM_train_dirs = get_AMM_train_dirs(linear_name, linear_name_full, method, feedback_bits, train_sam_num, test_sam_num)
train_data = np.load(os.path.join(AMM_train_dirs["dir_train"],AMM_train_dirs["linearin_path_train"]))

In [51]:
# 读取split
pkl_path = os.path.join(dir_now, 'CsiTransformerAMM/intermediate_var/pkl/%s/CKD(%i, %i, %i)_split_N%i.pkl' 
                        % (linear_name, ncodebooks, ncentroids[i_ncentroids], D, train_sam_num * batch_N_dict[linear_name]))
fr = open(pkl_path, 'rb')
splits = pickle.load(fr)
fr.close()
print(len(splits)) # 码本数
print(len(splits[0])) # 质心数以二为底的对数

64
7


In [52]:
# 读取bucket
pkl_path = os.path.join(dir_now, 'CsiTransformerAMM/intermediate_var/pkl/%s/CKD(%i, %i, %i)_bucket_N%i.pkl'
                        % (linear_name, ncodebooks, ncentroids[i_ncentroids], D, train_sam_num * batch_N_dict[linear_name]))
fr = open(pkl_path, 'rb')
buckets = pickle.load(fr)
fr.close()
print(len(buckets)) # 码本数
print(len(buckets[0])) # 质心数

64
128


In [53]:
# 查看第i_cb个码本splits的值
p=splits[i_cb]
for p_i in p:
    for v in p_i.vals:
        # 不带换行
        v_real = v/p_i.scaleby+ p_i.offset
        print(v_real, end=' ')
    #换行
    print()

0.4521618187427521 
0.23634111881256104 0.634778618812561 
0.06226803734898567 0.33570553734898567 0.5544555373489857 0.6403930373489857 
-0.1420036107301712 0.1548713892698288 0.2876838892698288 0.3970588892698288 0.5064338892698288 0.5923713892698288 -0.0013786107301712036 0.6392463892698288 
-0.35766254365444183 -0.021725043654441833 0.11108745634555817 0.19702495634555817 0.25952495634555817 0.31421245634555817 0.36889995634555817 0.42358745634555817 0.47827495634555817 0.5329624563455582 0.5642124563455582 0.6345249563455582 -0.0061000436544418335 -0.0061000436544418335 -0.0061000436544418335 0.6345249563455582 
-0.5455603301525116 -0.2408728301525116 -0.0768103301525116 0.024752169847488403 0.0872521698474884 0.1341271698474884 0.1810021698474884 0.2200646698474884 0.2513146698474884 0.2747521698474884 0.2981896698474884 0.3216271698474884 0.3528771698474884 0.3763146698474884 0.4075646698474884 0.4310021698474884 0.4622521698474884 0.4856896698474884 0.5169396698474884 0.5403771

In [54]:
buckets_cb = buckets[i_cb] # 第i_cb个码本的buckets
# for i, bucket in enumerate(buckets_cb):
#     # if len(bucket.point_ids) < 1: # 查看空的bucket
#     #     print("bucket %i" % i)
#     #     print(len(bucket.point_ids))
#     # 打印K个bucket中的存在的点的具体值
#     if len(bucket.point_ids) > 0:
#         print("bucket %i" % i)
#         print(len(bucket.point_ids))
# 保存K个bucket中的存在的点的具体值，至txt文件
last_max = -np.inf
max_i = -1
with open("bucket_point_(%i, %i, %i)_cb%i.txt" % (ncodebooks, ncentroids[i_ncentroids], D, i_cb), "w") as f:
    for i, bucket in enumerate(buckets_cb):
        if len(bucket.point_ids) > 0:
            f.write("bucket %i" % i)
            f.write("\n")
            f.write(str(len(bucket.point_ids)))
            f.write("\n")
            buc_min = np.inf
            buc_max = -np.inf
            for point_id in bucket.point_ids:
                point_val = train_data[point_id, i_cb]
                f.write(str(point_val))
                f.write(" ")
                if point_val < buc_min:
                    buc_min = point_val
                if point_val > buc_max:
                    buc_max = point_val
            f.write("\n")
            f.write("min: %f, max: %f, max-min: %f\n" % (buc_min, buc_max, buc_max-buc_min))
            if last_max > buc_min: # 如果之前的bucket有超过现在的bucket的值
                raise ValueError("last_max > buc_min: %f > %f, index: %i, %i" % (last_max, buc_min, max_i, i))
            if buc_max > last_max:
                last_max = buc_max # 前面所有bucket的最大值
                max_i = i
            f.write("col_sum_sqs:\n")
            for sum_sqs in bucket.col_sum_sqs():
                f.write(str(sum_sqs))
                f.write(" ")
            f.write("\n")
        else:
            print("bucket %i" % i) # 打印空的bucket
            print(len(bucket.point_ids)) # 打印空的bucket的长度
            f.write("bucket %i" % i)
            f.write("\n")
            f.write(str(len(bucket.point_ids)))
            f.write("\n")
            f.write("\n")
            f.write("\n")
            f.write("\n")
            f.write("\n")

        

bucket 85
0
bucket 89
0
bucket 90
0
bucket 91
0
bucket 97
0
bucket 98
0
bucket 99
0
bucket 100
0
bucket 101
0
bucket 102
0
bucket 103
0
bucket 104
0
bucket 105
0
bucket 106
0
bucket 107
0
bucket 108
0
bucket 109
0
bucket 110
0
bucket 111
0
bucket 113
0
bucket 114
0
bucket 115
0
bucket 116
0
bucket 117
0
bucket 118
0
bucket 119
0
bucket 121
0
bucket 122
0
bucket 123
0
bucket 125
0


In [55]:
i = 0
for p_x in p:
    # print(p_x.scaleby)
    print("x:", i, ", p_x.vals:", p_x.vals*256/p_x.scaleby)
    i += 1

x: 0 , p_x.vals: [178.]
x: 1 , p_x.vals: [150. 252.]
x: 2 , p_x.vals: [128. 198. 254. 276.]
x: 3 , p_x.vals: [102. 178. 212. 240. 268. 290. 138. 302.]
x: 4 , p_x.vals: [ 74. 160. 194. 216. 232. 246. 260. 274. 288. 302. 310. 328. 164. 164.
 164. 328.]
x: 5 , p_x.vals: [ 50. 128. 170. 196. 212. 224. 236. 246. 254. 260. 266. 272. 280. 286.
 294. 300. 308. 314. 322. 328. 332. 342. 352. 352. 188. 188. 188. 188.
 188. 188. 188. 352.]
x: 6 , p_x.vals: [ 30.  96. 134. 160. 182. 196. 210. 220. 228. 234. 240. 246. 252. 258.
 262. 266. 272. 274. 278. 280. 284. 288. 290. 294. 298. 300. 304. 308.
 312. 314. 318. 322. 326. 330. 332. 336. 340. 342. 346. 348. 352. 352.
 360. 360. 372. 208. 372. 372. 208. 208. 208. 208. 208. 208. 208. 208.
 208. 208. 208. 208. 208. 208. 208. 372.]


In [56]:
i = 0
for p_x in p:
    # print(p_x.scaleby)
    print("x:", i, ", p_x.scaleby:", p_x.scaleby)
    i += 1

x: 0 , p_x.scaleby: 128.0
x: 1 , p_x.scaleby: 128.0
x: 2 , p_x.scaleby: 128.0
x: 3 , p_x.scaleby: 128.0
x: 4 , p_x.scaleby: 128.0
x: 5 , p_x.scaleby: 128.0
x: 6 , p_x.scaleby: 128.0


In [57]:
base_i = 5 if ncentroids[i_ncentroids] < 256 else 6 # 2^base_i个split
print(f"当前显示分割阈值二叉树的第{base_i}层和它的子节点")
p_base = p[base_i].vals/ p[base_i].scaleby + p[base_i].offset
p_child = p[base_i+1].vals/ p[base_i+1].scaleby + p[base_i+1].offset
for i, p_base_item in enumerate(p_base):
    p_child_i = [2*i, 2*i+1]# p_base的每个split对应的孩子的index
    print("父节点：%.6f, 子节点%.6f,%.6f" % (p_base[i], p_child[p_child_i[0]], p_child[p_child_i[1]]))

当前显示分割阈值二叉树的第5层和它的子节点
父节点：-0.545560, 子节点-0.698831,-0.441019
父节点：-0.240873, 子节点-0.292581,-0.191019
父节点：-0.076810, 子节点-0.105081,-0.050394
父节点：0.024752, 子节点0.004294,0.043356
父节点：0.087252, 子节点0.074606,0.098044
父节点：0.134127, 子节点0.121481,0.144919
父节点：0.181002, 子节点0.168356,0.191794
父节点：0.220065, 子节点0.207419,0.223044
父节点：0.251315, 子节点0.246481,0.254294
父节点：0.274752, 子节点0.269919,0.277731
父节点：0.298190, 子节点0.293356,0.308981
父节点：0.321627, 子节点0.316794,0.332419
父节点：0.352877, 子节点0.348044,0.355856
父节点：0.376315, 子节点0.371481,0.387106
父节点：0.407565, 子节点0.402731,0.410544
父节点：0.431002, 子节点0.426169,0.441794
父节点：0.462252, 子节点0.457419,0.473044
父节点：0.485690, 子节点0.480856,0.496481
父节点：0.516940, 子节点0.512106,0.519919
父节点：0.540377, 子节点0.535544,0.543356
父节点：0.556002, 子节点0.558981,0.558981
父节点：0.595065, 子节点0.590231,0.590231
父节点：0.634127, 子节点0.637106,-0.003519
父节点：0.634127, 子节点0.637106,0.637106
父节点：-0.006498, 子节点-0.003519,-0.003519
父节点：-0.006498, 子节点-0.003519,-0.003519
父节点：-0.006498, 子节点-0.003519,-0.003519
父节点：-0.006498,

In [64]:
# 创建长度为ncentroids[i_ncentroids]-1的np数组，用于存储每个节点的split值
real_splits = np.zeros(ncentroids[i_ncentroids]-1, dtype=np.float32)
# print(real_splits.shape)
for i, p_x in enumerate(p):
    p_x_real_vals = p_x.vals/ p_x.scaleby + p_x.offset
    # print("x:", i, ", p_x.real_vals:", p_x.vals/ p_x.scaleby + p_x.offset)
    # 将树的每一层的p_x_real_vals添加到real_splits中
    real_splits[(2 ** i - 1):(2 ** (i+1) - 1)] = p_x_real_vals
# print(real_splits)

In [90]:
# Import Graph and EdgeSeq classes from the igraph module
from igraph import Graph, EdgeSeq

# Set the number of vertices to 25
nr_vertices = real_splits.shape[0]

# Create a list of vertex labels using the map function (Keep three significant digits)
v_label = list(map(lambda x: "%.3f" % x, real_splits))
# v_label = list(map(str, real_splits))

# Create a tree graph with 25 vertices and 2 children per vertex
G = Graph.Tree(nr_vertices, 2)

# Create a layout for the tree graph using the 'rt' (reingold_tilford) algorithm
lay = G.layout('tree')

# Create a dictionary of vertex positions from the layout
position = {k: lay[k] for k in range(nr_vertices)}
print(position)

# Create a list of Y-coordinates for the vertices
Y = [lay[k][1] for k in range(nr_vertices)]

# Find the maximum Y-coordinate of the vertices
M = max(Y)

# Create a sequence of edges from the graph
es = EdgeSeq(G)

# Create a list of edges from the edge sequence
E = [e.tuple for e in G.es]

# Get the length of the position dictionary
L = len(position)

# Create lists of X- and Y-coordinates for the vertices and edges
Xn = [position[k][0] for k in range(L)]
Yn = [2*M-position[k][1] for k in range(L)]
Xe = []
Ye = []

# Loop through the edges and add their X- and Y-coordinates to the corresponding lists
for edge in E:
    Xe+=[position[edge[0]][0],position[edge[1]][0], None]
    Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None]

# Set the labels variable to the list of vertex labels
labels = v_label


{0: [-35.40625, 5.0], 1: [-35.40625, 6.0], 2: [-24.15625, 4.0], 3: [-43.40625, 7.0], 4: [-27.40625, 7.0], 5: [-13.28125, 3.0], 6: [-12.90625, 5.0], 7: [-47.40625, 8.0], 8: [-39.40625, 8.0], 9: [-31.40625, 8.0], 10: [-23.40625, 8.0], 11: [-5.59375, 2.0], 12: [-2.40625, 4.0], 13: [-16.90625, 6.0], 14: [-8.90625, 6.0], 15: [-49.40625, 9.0], 16: [-45.40625, 9.0], 17: [-41.40625, 9.0], 18: [-37.40625, 9.0], 19: [-33.40625, 9.0], 20: [-29.40625, 9.0], 21: [-25.40625, 9.0], 22: [-21.40625, 9.0], 23: [2.09375, 3.0], 24: [-1.0, 1.0], 25: [-4.40625, 5.0], 26: [-0.40625, 5.0], 27: [-18.90625, 7.0], 28: [-14.90625, 7.0], 29: [-10.90625, 7.0], 30: [-6.90625, 7.0], 31: [-50.40625, 10.0], 32: [-48.40625, 10.0], 33: [-46.40625, 10.0], 34: [-44.40625, 10.0], 35: [-42.40625, 10.0], 36: [-40.40625, 10.0], 37: [-38.40625, 10.0], 38: [-36.40625, 10.0], 39: [-34.40625, 10.0], 40: [-32.40625, 10.0], 41: [-30.40625, 10.0], 42: [-28.40625, 10.0], 43: [-26.40625, 10.0], 44: [-24.40625, 10.0], 45: [-22.40625, 10

In [91]:
import plotly.graph_objects as go
import plotly as py
fig = go.Figure()
fig.add_trace(go.Scatter(x=Xe,
                   y=Ye,
                   mode='lines',
                   line=dict(color='rgb(210,210,210)', width=1),
                   hoverinfo='none'
                   ))
fig.add_trace(go.Scatter(x=Xn,
                  y=Yn,
                  mode='markers',
                  name='bla',
                  marker=dict(symbol='circle-dot',
                                size=20,
                                color='#6175c1',    #'#DB4551',
                                line=dict(color='rgb(50,50,50)', width=1)
                                ),
                  text=labels,
                  hoverinfo='text',
                  opacity=0.8
                  ))

In [92]:
def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
    L=len(pos)
    if len(text)!=L:
        raise ValueError('The lists pos and text must have the same len')
    annotations = []
    for k in range(L):
        annotations.append(
            dict(
                text=labels[k], # or replace labels with a different list for the text within the circle
                x=pos[k][0], y=2*M-position[k][1],
                xref='x1', yref='y1',
                font=dict(color=font_color, size=font_size),
                showarrow=False)
        )
    return annotations

In [93]:
axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
            zeroline=False,
            showgrid=False,
            showticklabels=False,
            )

fig.update_layout(title= 'Tree with Reingold-Tilford Layout',
              annotations=make_annotations(position, v_label),
              font_size=12,
              showlegend=False,
              xaxis=axis,
              yaxis=axis,
              margin=dict(l=40, r=40, b=85, t=100),
              hovermode='closest',
              plot_bgcolor='rgb(180,180,180)'
              )
py.offline.plot(fig, filename='split_tree_%s_C%i_icb%i_K%i_tr%i.html' % 
                (linear_name, ncodebooks, i_cb, ncentroids[i_ncentroids], train_sam_num))
fig.show()